In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}  # Cabeçalho padrão
SEARCH_VALUE = "smartphone"  # Parâmetro de pesquisa

In [ ]:
# Requisição para página usando o SEARCH_VALUE de pesquisa
session = requests.Session()
response = session.get("https://www.mercadolivre.com.br/", timeout=5, headers=HEADERS)
session.headers.update
response = session.get(
    f"https://lista.mercadolivre.com.br/{SEARCH_VALUE}#D[A:{SEARCH_VALUE}]", timeout=5
)

In [ ]:
# Coleta das informações (nome do produto, preço, avalição (nota e quantidade) e navegação de página)
soup = BeautifulSoup(response.text, "html.parser")
nome_produto = soup.findAll(class_="poly-box poly-component__title")
preco_produto = soup.findAll(class_="andes-money-amount__fraction")
avaliacao_produto = soup.findAll(class_="andes-visually-hidden")
nav_pagina = soup.findAll(class_="andes-pagination__link")

# Impressão das informações (nome do produto, preço, avalição (nota e quantidade) e navegação de página)
for i in nome_produto:
    print(i.text)

for i in preco_produto:
    print(i.text)

for i in avaliacao_produto:
    print(i.text)

for i in nav_pagina:
    print(i.get("href"))

In [ ]:
# Encerramento da conexão
session.close()